# MLflow

Dokumentacja: https://mlflow.org/docs/latest/getting-started/intro-quickstart/index.html

In [ ]:
#pip install mlflow

In [1]:
import pandas as pd 
from mlflow.transformers import generate_signature_output
import mlflow
from mlflow.models import infer_signature
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from mlflow import MlflowClient

In [2]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [3]:
# Ładowanie danych
df= pd.read_csv('data/Restaurant_revenue.csv')

In [ ]:
# zmienne numeryczne
num_features = df.select_dtypes(exclude = 'object').columns[:-1]
num_features

In [4]:
# Train/test split
train_x, test_x, train_y, test_y = train_test_split(df.drop(['Monthly_Revenue','Cuisine_Type'],axis=1),df['Monthly_Revenue'])

In [ ]:
# w wierszu poleceń: mlflow ui

In [5]:
# Ustawienie lokalnego adresu
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [ ]:
# Ustawienie nazwy eksperymentu
mlflow.set_experiment("Reastaurant revenue")

In [7]:
# Model i ewaluacja
min_samples_leaf  = 10
model = GradientBoostingRegressor(min_samples_leaf=min_samples_leaf).fit(train_x,train_y)
r2_train = model.score(train_x,train_y)
r2_test = model.score(test_x,test_y)

In [ ]:
# Logowanie modelu
with mlflow.start_run():
    mlflow.log_param('min_samples_leaf', min_samples_leaf)
    mlflow.log_metric('r2_train',r2_train)
    mlflow.log_metric('r2_test',r2_test)
    signature = infer_signature(train_x, model.predict(train_x))
    model_info = mlflow.sklearn.log_model(
        sk_model = model,
        artifact_path = 'restaurant_model',
        signature = signature,
        input_example = train_x,
        registered_model_name = "restaurant_model_v1"
    )

In [9]:
# Druga wersja
min_samples_leaf  = 20
model = GradientBoostingRegressor(min_samples_leaf=min_samples_leaf).fit(train_x,train_y)
r2_train = model.score(train_x,train_y)
r2_test = model.score(test_x,test_y)

In [ ]:
# Logowanie nowej wersji
with mlflow.start_run():
    mlflow.log_param('min_samples_leaf', min_samples_leaf)
    mlflow.log_metric('r2_train',r2_train)
    mlflow.log_metric('r2_test',r2_test)
    signature = infer_signature(train_x, model.predict(train_x))
    model_info = mlflow.sklearn.log_model(
        sk_model = model,
        artifact_path = 'restaurant_model',
        signature = signature,
        input_example = train_x,
        registered_model_name = "restaurant_model_v2"
    )

In [ ]:
# Logowanie nowej wersji
with mlflow.start_run():
    mlflow.log_param('min_samples_leaf', min_samples_leaf)
    mlflow.log_metric('r2_train',r2_train)
    mlflow.log_metric('r2_test',r2_test)
    signature = infer_signature(train_x, model.predict(train_x))
    model_info = mlflow.sklearn.log_model(
        sk_model = model,
        artifact_path = 'restaurant_model',
        signature = signature,
        input_example = train_x,
        registered_model_name = "restaurant_model_v1"
    )

In [ ]:
model_info.model_uri

In [ ]:
# ładowanie modelu 
loaded_model  =mlflow.pyfunc.load_model(model_info.model_uri)

In [22]:
# Predykcje
preds = loaded_model.predict(test_x)

In [ ]:
loaded_model_2 = mlflow.sklearn.load_model(model_info.model_uri)

In [ ]:
loaded_model_2

In [ ]:
loaded_model_2.feature_names_in_

In [ ]:
# dodanie opisu
client = MlflowClient()
client.update_model_version(name = 'restaurant_model_v1',
                            version =1 ,
                            description='This is the first version of the model.')

In [30]:
# Zmiana nazwy modelu
client.rename_registered_model(name = 'restaurant_model_v1',
                               new_name= 'restaurant_model_production')

In [31]:
# Dodanie tagów
client.set_registered_model_tag('restaurant_model_production','env','PROD')